# Background on Taproot

## Where does "taproot" come from?

Taproot is a series of bitcoin BIPS (bitcoin improvement proposals) that were merged into bitcoin-core in 2021.

It was most recent bitcoin soft-fork and it went live in November 2021.

The BIPS can be found on github. They are as follows:

- BIP340, Schnorr Signatures https://github.com/bitcoin/bips/blob/master/bip-0340.mediawiki
- BIP341, Taproot Segwit v1 https://github.com/bitcoin/bips/blob/master/bip-0341.mediawiki
- BIP342, Validating Taproot Spends https://github.com/bitcoin/bips/blob/master/bip-0342.mediawiki
- BIP350, Bech32m https://github.com/bitcoin/bips/blob/master/bip-0350.mediawiki

## What did "taproot" change?

Taproot added and modified many things in bitcoin. Including

- A new signature algorithm (Schnorr)
- A new version of Script opcodes (Segwit v1)
- A new bitcoin address checksum (bech32m)
- Removed some opcodes we know and love (OP_CHECKMULTISIG)
- Added a new opcode (OP_CHECKSIGADD)
- Added a new scripthash algorithm

# Finding Taproot in the Wild

Let's go to mempool.space and find some taproot addresses in the wild!



## How to spot a taproot address

The bech32 address has a `p` immediately following the `1`

Find a taproot address on mempool dot space.

What's the "Type" for this address?


Copy paste the "ScriptPubKey (HEX)" for the address you found into the pyblock below.

In [2]:
scriptpubkey = ""

In [3]:
print(scriptpubkey)

51208fe72845dafd109cefa18fe63c75a3df7c828c2cebd5eb46024dd86ee2ad9b21


This is a series of opcodes. If you pass this to `bitcoin-cli decodescript` what does it print back as the asm?

In [5]:
output = {
  "asm": "1 8fe72845dafd109cefa18fe63c75a3df7c828c2cebd5eb46024dd86ee2ad9b21",
  "desc": "rawtr(8fe72845dafd109cefa18fe63c75a3df7c828c2cebd5eb46024dd86ee2ad9b21)#3hn678ar",
  "address": "bcrt1p3lnjs3w6l5gfemap3lnrcadrma7g9rpva027k3szfhvxac4dnvss6rwn89",
  "type": "witness_v1_taproot"
}

In [6]:
print(output['asm'])

1 8fe72845dafd109cefa18fe63c75a3df7c828c2cebd5eb46024dd86ee2ad9b21


In [8]:
print(output['type'])

witness_v1_taproot


## Let's Get Familiar With The Tools!

In [30]:
from codes import parse_tx_bytes

tx = ""

In [31]:
print(parse_tx_bytes(tx))

IndexError: index out of range

In [32]:
from pprint import pprint

In [33]:
pprint(parse_tx_bytes(tx))

IndexError: index out of range

First things first, let's update the `parse_tx_bytes` method to be able to handle segwit (any version) transactions.

In [28]:
from codes import parse_compact_size, parse_input_bytes, parse_output_bytes

def parse_tx_bytes_mine(tx_hex):
  tx_bytes = bytes.fromhex(tx_hex)

  tx = {}
  ptr = 0
  tx['version'] = tx_bytes[0:4]
  ptr += 4

  if tx_bytes[ptr] == 0x00:
    assert tx_bytes[ptr+1] == 0x01
    tx['marker_flag'] = bytes([0x00, 0x01])
    ptr += 2

  count, size = parse_compact_size(tx_bytes[ptr:])
  ptr += size 
  tx['inputs'] = []
  for _ in range(0, count):
    inputx, size = parse_input_bytes(tx_bytes[ptr:])
    ptr += size
    tx['inputs'].append(inputx)
 
  count, size = parse_compact_size(tx_bytes[ptr:])
  ptr += size
  tx['outputs'] = []
  for _ in range(0, count):
    outputx, size = parse_output_bytes(tx_bytes[ptr:])
    ptr += size
    tx['outputs'].append(outputx)

  if 'marker_flag' in tx:
    tx['witnesses'] = []
    for _ in range(0, len(tx['inputs'])):
        witness, size = parse_witness_bytes(tx_bytes[ptr:])
        ptr += size
        tx['witnesses'].append(witness)

  tx['locktime'] = tx_bytes[ptr:]
  return tx

In [23]:
tx_again = "02000000000101bcf90bb3d161537bed5a5c2eecbbff1c525acebfc6bd46cfb98573fd49ed06311903000000fffffffd014a010000000000002251208fe72845dafd109cefa18fe63c75a3df7c828c2cebd5eb46024dd86ee2ad9b21034077d25c310bf96107938952de14d80bec0639c158904acd16c6bab8d4bdfbf7f30b25856cbfa79df5f22f1e4f2047cc7840bd5afec6b4fe66a7eef7c9e178a1da6b20b4a6bf508005c4a95fa7494da1d1b756ac225b1ba160bfb4087db71b4bfd779cac0063036f726401010a746578742f706c61696e00337b2270223a226272632d3230222c226f70223a226d696e74222c227469636b223a2262656172222c22616d74223a223130227d6821c1b4a6bf508005c4a95fa7494da1d1b756ac225b1ba160bfb4087db71b4bfd779c00000000"

In [37]:
pprint(parse_tx_bytes_mine(tx_again))

{'inputs': [{'scriptSig': b'',
             'sequence': b'\xff\xff\xff\xfd',
             'txid': b'\xbc\xf9\x0b\xb3\xd1aS{\xedZ\\.\xec\xbb\xff\x1c'
                     b'RZ\xce\xbf\xc6\xbdF\xcf\xb9\x85s\xfdI\xed\x061',
             'vout': b'\x19\x03\x00\x00'}],
 'locktime': b'@w\xd2\\1\x0b\xf9a\x07\x93\x89R\xde\x14\xd8\x0b\xec\x069\xc1'
             b'X\x90J\xcd\x16\xc6\xba\xb8\xd4\xbd\xfb\xf7\xf3\x0b%\x85'
             b'l\xbf\xa7\x9d\xf5\xf2/\x1eO G\xccx@\xbdZ\xfe\xc6\xb4\xfe'
             b'f\xa7\xee\xf7\xc9\xe1x\xa1\xdak \xb4\xa6\xbfP\x80\x05\xc4\xa9_'
             b'\xa7IM\xa1\xd1\xb7V\xac"[\x1b\xa1`\xbf\xb4\x08}\xb7\x1bK'
             b'\xfdw\x9c\xac\x00c\x03ord\x01\x01\ntext/plain\x003{"p":"brc-2'
             b'0","op":"mint","tick":"bear","amt":"10"}h!\xc1\xb4\xa6\xbfP\x80'
             b'\x05\xc4\xa9_\xa7IM\xa1\xd1\xb7V\xac"[\x1b\xa1`\xbf\xb4\x08'
             b'}\xb7\x1bK\xfdw\x9c\x00\x00\x00\x00',
 'marker_flag': b'\x00\x01',
 'outputs': [{'amount': b'J\x01\x00\x00\x00\x

In [36]:
def parse_witness_bytes(data):
    return {'elements': []}, 1